In [1]:
from elasticsearch import Elasticsearch
import requests

es = Elasticsearch([{"host": "localhost", "port": 9200}])

In [94]:
res = es.search(index="online_courses", 
                body={
                    "_source": [
                        "context.user_id",
                        "context.course_id",
                        "time",
                        "event.submission",
                        "event.attempts",
                        "event.problem_id" 
                    ],
                    "query": {                       
                          
                                "bool":{
                                    "must":{
                                        "match_phrase":{
                                            "event_type": "problem_check"
                                        }
                                    },
                                    "must_not":{
                                        "match_phrase":{
                                            "event_type": "problem_check_fail"
                                        }
                                    }, 
                                    "filter":[{
                                        "match_phrase":{
                                            "event_source": "server"
                                        }
                                    }]
                                }
                        
                            }
                        
                    
                }
               )

In [95]:
res

{'took': 201,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': 226,
  'max_score': 3.7074828,
  'hits': [{'_index': 'online_courses',
    '_type': 'courses',
    '_id': '4XOE2GkBXJZRjjpWbgpM',
    '_score': 3.7074828,
    '_source': {'context': {'course_id': 'course-v1:spbu+SOCLAB+spring_2018',
      'user_id': 492510},
     'time': '2018-02-15T12:45:45.356044+00:00'}},
   {'_index': 'online_courses',
    '_type': 'courses',
    '_id': 'lnOE2GkBXJZRjjpWbhNd',
    '_score': 3.7074828,
    '_source': {'context': {'course_id': 'course-v1:spbu+SOCLAB+spring_2018',
      'user_id': 486614},
     'time': '2018-02-18T12:21:36.318403+00:00'}},
   {'_index': 'online_courses',
    '_type': 'courses',
    '_id': '1nOE2GkBXJZRjjpWbhNd',
    '_score': 3.7074828,
    '_source': {'context': {'course_id': 'course-v1:spbu+SOCLAB+spring_2018',
      'user_id': 486614},
     'time': '2018-02-18T12:30:17.686613+00:00'}},
   {'_index': 'online_co

In [ ]:
                                    "must_not":{
                                        "match_phrase":{
                                            "event_type": "problem_check_fail"
                                        }
                                    } 

In [83]:
len(res["hits"]["hits"])

10